# Pandas I

Adapted from [Chris Fonnesbeck](https://github.com/fonnesbeck/statistical-analysis-python-tutorial)

**pandas** is a Python package providing fast, flexible, and expressive data structures designed to work with both *relational* and *labeled* data. It is a fundamental high-level building block for doing practical, real world data analysis in Python. 

### pandas is well suited for:

- Tabular data with heterogeneously-typed columns, as in an SQL table or Excel spreadsheet
- Ordered and unordered (not necessarily fixed-frequency) time series data.
- Arbitrary matrix data (homogeneously typed or heterogeneous) with row and column labels
- Any other form of observational / statistical data sets. The data actually need not be labeled at all to be placed into a pandas data structure

### Key features:
    
- Easy handling of **missing data**
- **Size mutability**: columns can be inserted and deleted from DataFrame and higher dimensional objects
- Automatic and explicit **data alignment**: objects can be explicitly aligned to a set of labels, or the data can be aligned automatically
- Powerful, flexible **group by functionality** to perform split-apply-combine operations on data sets
- Intelligent label-based **slicing, fancy indexing, and subsetting** of large data sets
- Intuitive **merging and joining** data sets
- Flexible **reshaping and pivoting** of data sets
- **Hierarchical labeling** of axes
- Robust **IO tools** for loading data from flat files, Excel files, databases, and HDF5
- **Time series functionality**: date range generation and frequency conversion, moving window statistics, moving window linear regressions, date shifting and lagging, etc.

In [ ]:
from IPython.core.display import HTML
HTML("<iframe src=http://pandas.pydata.org width=1024 height=500></iframe>")

### Some basic setup

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np

## Pandas Data Structures

### Series

A **Series** is a single vector of data with an *index* that labels each element in the vector

In [ ]:
counts = pd.Series([632, 1638, 569, 115])
counts

If an index is not specified, a default sequence of integers is assigned as the index. A NumPy array comprises the values of the `Series`, while the index is a pandas `Index` object.

In [ ]:
counts.values

In [ ]:
counts.index

We can assign meaningful labels to the index, if they are available:

In [ ]:
bacteria = pd.Series([632, 1638, 569, 115], 
    index=['Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes'])

In [ ]:
bacteria

These labels can be used to refer to the values in the `Series`.

In [ ]:
bacteria['Actinobacteria']

In [ ]:
bacteria[[name.endswith('bacteria') for name in bacteria.index]]

In [ ]:
[name.endswith('bacteria') for name in bacteria.index]

Notice that the indexing operation preserved the association between the values and the corresponding indices.

We can still use positional indexing if we wish.

In [ ]:
bacteria[0]

We can give both the array of values and the index meaningful labels themselves:

In [ ]:
bacteria.name = 'counts'
bacteria.index.name = 'phylum'
bacteria

We can also filter according to the values in the `Series`:

In [ ]:
bacteria[bacteria>1000]

A `Series` can be thought of as an ordered key-value store. In fact, we can create one from a `dict`:

In [ ]:
bacteria_dict = {'Firmicutes': 632, 'Proteobacteria': 1638, 'Actinobacteria': 569, 'Bacteroidetes': 115}
pd.Series(bacteria_dict)

Notice that the `Series` is created in key-sorted order.

If we pass a custom index to `Series`, it will select the corresponding values from the dict, and treat indices without corrsponding values as missing. Pandas uses the `NaN` (not a number) type for missing values.

In [ ]:
bacteria2 = pd.Series(bacteria_dict, index=['Cyanobacteria','Firmicutes','Proteobacteria','Actinobacteria'])
bacteria2

In [ ]:
bacteria2.isnull()

Critically, the labels are used to **align data** when used in operations with other Series objects:

In [ ]:
bacteria + bacteria2

### DataFrame

Inevitably, we want to be able to store, view and manipulate data that is *multivariate*, where for every index there are multiple fields or columns of data, often of varying data type.

A `DataFrame` is a tabular data structure, encapsulating multiple series like columns in a spreadsheet. Data are stored internally as a 2-dimensional object, but the `DataFrame` allows us to represent and manipulate higher-dimensional data.

In [ ]:
data = pd.DataFrame({'value':[632, 1638, 569, 115, 433, 1130, 754, 555],
                     'patient':[1, 1, 1, 1, 2, 2, 2, 2],
                     'phylum':['Firmicutes', 'Proteobacteria', 'Actinobacteria', 
                               'Bacteroidetes', 'Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes']})

In [ ]:
data

Notice the `DataFrame` is sorted by column name. We can change the order by indexing them in the order we desire:

In [ ]:
data[['phylum','value','patient']]

A `DataFrame` has a second index, representing the columns:

In [ ]:
data.columns

If we wish to access columns, we can do so either by dict-like indexing or by attribute:

In [ ]:
data['value']

In [ ]:
data.value

In [ ]:
type(data.value)

In [ ]:
type(data[['value']])

Notice this is different than with `Series`, where dict-like indexing retrieved a particular element (row). If we want access to a row in a `DataFrame`, we index its `ix` attribute.


In [ ]:
data.ix[3]

Alternatively, we can create a `DataFrame` with a dict of dicts:

In [ ]:
data = pd.DataFrame({0: {'patient': 1, 'phylum': 'Firmicutes', 'value': 632},
                     1: {'patient': 1, 'phylum': 'Proteobacteria', 'value': 1638},
                     2: {'patient': 1, 'phylum': 'Actinobacteria', 'value': 569},
                     3: {'patient': 1, 'phylum': 'Bacteroidetes', 'value': 115},
                     4: {'patient': 2, 'phylum': 'Firmicutes', 'value': 433},
                     5: {'patient': 2, 'phylum': 'Proteobacteria', 'value': 1130},
                     6: {'patient': 2, 'phylum': 'Actinobacteria', 'value': 754},
                     7: {'patient': 2, 'phylum': 'Bacteroidetes', 'value': 555}})

In [ ]:
data

We probably want this transposed:

In [ ]:
data = data.T

In [ ]:
data

Its important to note that the Series returned when a DataFrame is indexted is merely a **view** on the DataFrame, and not a copy of the data itself. So you must be cautious when manipulating this data:

In [ ]:
vals = data.value
vals

In [ ]:
vals[5] = 0
vals

In [ ]:
data

In [ ]:
vals = data.value.copy()
vals[5] = 1000
data

We can create or modify columns by assignment:

In [ ]:
data.value[3] = 14
data

In [ ]:
data['year'] = 2013
data

But note, we cannot use the attribute indexing method to add a new column:

In [ ]:
data.treatment = 1
data

In [ ]:
data.treatment

Specifying a `Series` as a new columns cause its values to be added according to the `DataFrame`'s index:

In [ ]:
treatment = pd.Series([0]*4 + [1]*2)
treatment

In [ ]:
data['treatment'] = treatment
data

Other Python data structures (ones without an index) need to be the same length as the `DataFrame`:

In [ ]:
month = ['Jan', 'Feb', 'Mar', 'Apr']
data['month'] = month

In [ ]:
data['month'] = ['Jan']*len(data)
data

We can use `del` to remove columns, in the same way `dict` entries can be removed:

In [ ]:
del data['month']
data

Or `.drop()` can be used:

In [ ]:
data.drop?

In [ ]:
data['month'] = ['Jan']*len(data)
data.drop('month', axis=1, inplace=True)
data

We can extract the underlying data as a simple `ndarray` by accessing the `values` attribute:

In [ ]:
data.values

Notice that because of the mix of string and integer (and `NaN`) values, the dtype of the array is `object`. The dtype will automatically be chosen to be as general as needed to accomodate all the columns.

In [ ]:
df = pd.DataFrame({'foo': [1,2,3], 'bar':[0.4, -1.0, 4.5]})
df.values

Pandas uses a custom data structure to represent the indices of Series and DataFrames.

In [ ]:
data.index

Index objects are immutable:

In [ ]:
data.index[0] = 15

This is so that Index objects can be shared between data structures without fear that they will be changed.

In [ ]:
bacteria2.index = bacteria.index

In [ ]:
bacteria2

## Importing data

Pandas provides a convenient set of functions for importing tabular data in a number of formats directly into a `DataFrame` object. These functions include a slew of options to perform type inference, indexing, parsing, iterating and cleaning automatically as data are imported.

Let's start with some more bacteria data, stored in csv format.

In [ ]:
!head -n10 data/microbiome.csv

This table can be read into a DataFrame using `read_csv`:

In [ ]:
mb = pd.read_csv("data/microbiome.csv")
mb

In [ ]:
mb.head()

In [ ]:
mb.tail()

Notice that `read_csv` automatically considered the first row in the file to be a header row.

We can override default behavior by customizing some the arguments, like `header`, `names` or `index_col`.

In [ ]:
pd.read_csv("data/microbiome.csv", header=None).head()

`read_csv` is just a convenience function for `read_table`, since csv is such a common format:

In [ ]:
mb = pd.read_table("data/microbiome.csv", sep=',')

The `sep` argument can be customized as needed to accomodate arbitrary separators. For example, we can use a regular expression to define a variable amount of whitespace, which is unfortunately very common in some data formats: 
    
    sep='\s+'

For a more useful index, we can specify the first two columns, which together provide a unique index to the data.

In [ ]:
mb = pd.read_csv("data/microbiome.csv", index_col=['Taxon','Patient'])
mb.head()

This is called a *hierarchical* index, which we will revisit later.

If we have sections of data that we do not wish to import (for example, known bad data), we can populate the `skiprows` argument:

In [ ]:
pd.read_csv("data/microbiome.csv", skiprows=[3,4,6]).head()

Conversely, if we only want to import a small number of rows from, say, a very large data file we can use `nrows`:

In [ ]:
pd.read_csv("data/microbiome.csv", nrows=4)

Alternately, if we want to process our data in reasonable chunks, the `chunksize` argument will return an iterable object that can be employed in a data processing loop. For example, our microbiome data are organized by bacterial phylum, with 15 patients represented in each:

In [ ]:
data_chunks = pd.read_csv("data/microbiome.csv", chunksize=15)

mean_tissue = {chunk.Taxon[0]: chunk.Tissue.mean() for chunk in data_chunks}
    
mean_tissue

Most real-world data is incomplete, with values missing due to incomplete observation, data entry or transcription error, or other reasons. Pandas will automatically recognize and parse common missing data indicators, including `NA` and `NULL`.

In [ ]:
!head -n12 data/microbiome_missing.csv

In [ ]:
pd.read_csv("data/microbiome_missing.csv").head(12)

Above, Pandas recognized `NA` and an empty field as missing data.

In [ ]:
pd.isnull(pd.read_csv("data/microbiome_missing.csv")).head(12)

Unfortunately, there will sometimes be inconsistency with the conventions for missing data. In this example, there is a question mark "?" and a large negative number where there should have been a positive integer. We can specify additional symbols with the `na_values` argument:
   

In [ ]:
pd.read_csv("data/microbiome_missing.csv", na_values=['?', -99999]).head(12)

These can be specified on a column-wise basis using an appropriate dict as the argument for `na_values`.

### Microsoft Excel

Since so much scientific data ends up in Excel spreadsheets, Pandas' ability to directly import Excel spreadsheets is valuable. This support is contingent on having one or two dependencies (depending on what version of Excel file is being imported) installed: `xlrd` and `openpyxl` (these may be installed with either `pip` or `easy_install`).

Importing Excel data to Pandas is a two-step process. First, we create an `ExcelFile` object using the path of the file:                                             

In [ ]:
mb_file = pd.ExcelFile('data/microbiome/MID1.xls')
mb_file

Then, since modern spreadsheets consist of one or more "sheets", we parse the sheet with the data of interest:

In [ ]:
mb1 = mb_file.parse("Sheet 1", header=None)
mb1.columns = ["Taxon", "Count"]
mb1.head()

There is now a `read_excel` conveneince function in Pandas that combines these steps into a single call:

In [ ]:
mb2 = pd.read_excel('data/microbiome/MID2.xls', sheetname='Sheet 1', header=None)
mb2.head()

There are many other types of files that Pandas can open, which we'll cover further in the future.

In [ ]:
HTML("<iframe src=http://pandas.pydata.org/pandas-docs/stable/io.html width=1024 height=500></iframe>")

## Pandas Fundamentals

This section introduces the new user to the key functionality of Pandas that is required to use the software effectively.

In [ ]:
baseball = pd.read_csv("data/baseball.csv", index_col='id')
baseball.head()

Notice that we specified the `id` column as the index, since it appears to be a unique identifier. We could try to create a unique index ourselves by combining `player` and `year`:

In [ ]:
player_id = baseball.player + baseball.year.astype(str)
baseball_newind = baseball.copy()
baseball_newind.index = player_id
baseball_newind.head()

This looks okay, but let's check:

In [ ]:
baseball_newind.index.is_unique

So, indices need not be unique. Our choice is not unique because some players change teams within years.

In [ ]:
pd.Series(baseball_newind.index).value_counts().head(15)

The most important consequence of a non-unique index is that indexing by label will return multiple values for some labels:

In [ ]:
baseball_newind.ix['wickmbo012007']

We can create a truly unique index by combining `player`, `team` and `year`:

In [ ]:
player_unique = baseball.player + baseball.team + baseball.year.astype(str)
baseball_newind = baseball.copy()
baseball_newind.index = player_unique
baseball_newind.head()

In [ ]:
baseball_newind.index.is_unique

We can create meaningful indices more easily using a hierarchical index; for now, we will stick with the numeric `id` field as our index.

### Manipulating indices

**Reindexing** allows users to manipulate the data labels in a DataFrame. It forces a DataFrame to conform to the new index, and optionally, fill in missing data if requested.

A simple use of `reindex` is to alter the order of the rows:

In [ ]:
baseball.reindex(baseball.index[::-1]).head()

Notice that the `id` index is not sequential. Say we wanted to populate the table with every `id` value. We could specify and index that is a sequence from the first to the last `id` numbers in the database, and Pandas would fill in the missing data with `NaN` values:

In [ ]:
id_range = range(baseball.index.values.min(), baseball.index.values.max())
baseball.reindex(id_range).head()

Missing values can be filled as desired, either with selected values, or by rule:

In [ ]:
baseball.reindex(id_range, method='ffill', columns=['player','year']).head()

In [ ]:
baseball.reindex(id_range, fill_value='mr.nobody', columns=['player']).head()

Keep in mind that `reindex` does not work if we pass a non-unique index series.

We can remove rows or columns via the `drop` method:

In [ ]:
baseball.shape

In [ ]:
baseball.drop([89525, 89526]).shape

In [ ]:
baseball.head()

In [ ]:
baseball.drop(['ibb','hbp','lg','g','ab'], axis=1).head()

## Indexing and Selection

Indexing works analogously to indexing in lists or NumPy arrays, except we can use the labels in the `Index` object to extract values in addition to arrays of integers.

In [ ]:
# Sample Series object
hits = baseball_newind.h
hits.head()

In [ ]:
# list/numpy-style indexing
hits[:3]

In [ ]:
# Indexing by label
hits[['womacto01CHN2006','schilcu01BOS2006']]

We can also slice with data labels, since they have an intrinsic order within the Index:

In [ ]:
hits['womacto01CHN2006':'gonzalu01ARI2006']

In [ ]:
hits['womacto01CHN2006':'gonzalu01ARI2006'] = 5
hits.head(12)

In a `DataFrame` we can slice along either or both axes:

In [ ]:
baseball_newind[['h','ab']].head()

In [ ]:
baseball_newind[baseball_newind.ab > 500].head()

The indexing field [`ix`](http://pandas.pydata.org/pandas-docs/stable/indexing.html#different-choices-for-indexing) allows us to select subsets of rows and columns in an intuitive way:

In [ ]:
baseball_newind.ix['gonzalu01ARI2006', ['h','X2b', 'X3b', 'hr']]

In [ ]:
baseball_newind.ix[['gonzalu01ARI2006','finlest01SFN2006'], 5:8]

In [ ]:
baseball_newind.ix[:'myersmi01NYA2006', 'hr']

## Operations

`DataFrame` and `Series` objects allow for several operations to take place either on a single object, or between two or more objects.

For example, we can perform arithmetic on the elements of two objects, such as combining baseball statistics across years:

In [ ]:
hr2006 = baseball.ix[baseball.year==2006,'hr']
hr2006.index = baseball.player[baseball.year==2006]
hr2006.head()

In [ ]:
hr2007 = baseball.ix[baseball.year==2007,'hr']
hr2007.index = baseball.player[baseball.year==2007]
hr2007.head()

In [ ]:
hr_total = hr2006 + hr2007
hr_total.head(20)

Pandas' data alignment places `NaN` values for labels that do not overlap in the two Series. In fact, there are only 6 players that occur in both years.

In [ ]:
hr_total[hr_total.notnull()]

In [ ]:
hr_total.dropna()

While we do want the operation to honor the data labels in this way, we probably do not want the missing values to be filled with `NaN`. We can use the `add` method to calculate player home run totals by using the `fill_value` argument to insert a zero for home runs where labels do not overlap:

In [ ]:
hr2007.add(hr2006, fill_value=0).head()

Operations can also be **broadcast** between rows or columns.

For example, if we subtract the maximum number of home runs hit from the `hr` column, we get how many fewer than the maximum were hit by each player:

In [ ]:
baseball.hr - baseball.hr.max()

We can also apply functions to each column or row of a `DataFrame`

In [ ]:
stats.head()

In [ ]:
stats.apply(np.median)

In [ ]:
stats.apply(lambda x: x.max() - x.min())

Lets use apply to calculate a meaningful baseball statistics, slugging percentage:

$$SLG = \frac{1B + (2 \times 2B) + (3 \times 3B) + (4 \times HR)}{AB}$$

In [ ]:
slg = lambda x: (x['h']-x['X2b']-x['X3b']-x['hr'] + 2*x['X2b'] + 3*x['X3b'] + 4*x['hr'])/(x['ab']+1e-6)
baseball.apply(slg, axis=1).apply(lambda x: '%.3f' % x).head()

## Data summarization

We often wish to summarize data in `Series` or `DataFrame` objects, so that they can more easily be understood or compared with similar data. The NumPy package contains several functions that are useful here, but several summarization or reduction methods are built into Pandas data structures.

### Describe
`.describe()` gives some useful statistics for each variable

In [ ]:
baseball.describe()

Some other useful summary measures include:

- `.sum()`
- `.mean()`
- `.corr()`
- etc

In [ ]:
baseball.mean()

The important difference between NumPy's functions and Pandas' methods is that the latter have built-in support for handling missing data.

In [ ]:
bacteria2

In [ ]:
bacteria2.mean()

Sometimes we may not want to ignore missing values, and allow the `nan` to propagate.

In [ ]:
bacteria2.mean(skipna=False)

Passing `axis=1` will summarize over rows instead of columns, which only makes sense in certain situations.

In [ ]:
extra_bases = baseball[['X2b','X3b','hr']].sum(axis=1)
extra_bases.sort_values(ascending=False).head()

We can also calculate summary statistics *across* multiple columns, for example, correlation and covariance.

$$cov(x,y) = \sum_i (x_i - \bar{x})(y_i - \bar{y})$$

In [ ]:
baseball.hr.cov(baseball.X2b)

$$corr(x,y) = \frac{cov(x,y)}{(n-1)s_x s_y} = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_i (x_i - \bar{x})^2 \sum_i (y_i - \bar{y})^2}}$$

In [ ]:
baseball.hr.corr(baseball.X2b)

In [ ]:
baseball.ab.corr(baseball.h)

In [ ]:
baseball.corr()

If we have a `DataFrame` with a hierarchical index (or indices), summary statistics can be applied with respect to any of the index levels:

In [ ]:
mb.head()

In [ ]:
mb.sum(level='Taxon')

## Exercise 3

Open up [Lecture 3/Exercise 3.ipynb](./Exercise 3.ipynb) in your Jupyter notebook server.

Solutions are at [Lecture 3/Exercise 3 - Solutions.ipynb](./Exercise 3 - Solutions.ipynb)